### BONUS FUNCTIONS:
- didn't get to in class

In [ ]:
def get_all_links(soup):#,attr_kwds=None):
    """Finds all links inside of soup that have the attributes(attr_kwds),which will be used in soup.findAll(attrs=attr_kwds).
    Returns a list of links.
    tag_type = 'a' or 'href'"""
    all_a_tags = soup.findAll('a',attrs=kwds) 
    link_list = []
    for link in all_a_tags:
        test_link = link.get('href')#,attr=kwds)
#         test_link = link.get('href',attrs=kwds)
        link_list.append(test_link)
    return link_list

In [ ]:
def make_absolute_links(source_url, rel_link_list):
    """Accepts the source_url for the source page of the rel_link_list and uses urljoin to return a list of valid absolute links."""
    
    from urllib.parse import urlparse, urljoin

    absolute_links=[]

    # Create a for loop to loop through links and make absolute html paths
    for link in rel_link_list:

        # Get base url using a url pasers and the story_url at the beginning of the nb
        abs_link = urljoin(source_url,link)    

        #concatenate and append to a list 
        absolute_links.append(abs_link)
    
    return absolute_links

#### Ex Functions


- `soup = cook_soup_from_url(url)`
    - make a beautiful soup from url
-`soup_links = get_all_links(soup)`
    - get all links from soup and return as a list.
    
-  `absolute_links = make_absolute_links(url, soup_links) `
    - use If `soup_links` are relative links that do not include the website domain and start with '../' instead of 'https://www... ').
    - then can use the `absolute_links` to make new soups to continue searching for your desired content.


In [ ]:
def cook_soup_from_url(url, parser='lxml',sleep_time=0):
    """Uses requests to retreive webpage and returns a BeautifulSoup made using lxml parser."""
    import requests
    from time import sleep
    from bs4 import BeautifulSoup
    
    sleep(sleep_time)
    response = requests.get(url)
    
    # check status of request
    if response.status_code != 200:
        raise Exception(f'Error: Status_code !=200.\n status_code={response.status_code}')
                        
    c = response.content
    # feed content into a beautiful soup using lxml
    soup = BeautifulSoup(c,'lxml')
    return soup

In [ ]:
def cook_batch_of_soups(link_list, sleep_time=1): #,user_fun = extract_target_text):
    """Accepts a list of links to extract and save in a list of dictionaries of soups
    with their relative url path as their key.
    Set user_fun to None to just extract full soups without user_extract"""
    from time import sleep
    from urllib.parse import urlparse, urljoin

    batch_of_soups = []
    
    for link in link_list:
        soup_dict = {}
        
        
        # turn the url path into the dictionary key/title
        url_dict_key_path = urlparse(link).path
        url_dict_key = url_dict_key_path.split('/')[-1]
        
        soup_dict['_url'] = link
        soup_dict['path'] = url_dict_key

        # make a soup from the current link
        page_soup = cook_soup_from_url(link, sleep_time=sleep_time)
        soup_dict['soup'] = page_soup

        
#         if user_fun!=None:
#             ## ADDING USER-SPECIFIED EXTRACTION FUNCTION       
#             user_output = user_fun(page_soup) #can add inputs to function
#             soup_dict['user_extract'] = user_output
        
        # Add current page's soup to batch_of_soups list
        batch_of_soups.append(soup_dict)
        
    return batch_of_soups


def extract_target_text(soup_or_tag,tag_name='p', attrs_dict=None, join_text =True, save_files=False):
    """User-specified function to add extraction of specific content during 'cook batch of soups'"""
    
    if attrs_dict==None:
        found_tags = soup_or_tag.find_all(name=tag_name)
    else:
        found_tags = soup_or_tag.find_all(name=tag_name,attrs=attrs_dict)
    
    
    # if extracting from multiple tags
    output=[]
    output = [tag.text for tag in found_tags if tag.text is not None]
    
    if join_text == True:
        output = ' '.join(output)

    ## ADDING SAVING EACH 
    if save_files==True:
        text = output #soup.body.string
        filename =f"drive/My Drive/text_extract_{url_dict_key}.txt"
        soup_dict['filename'] = filename
        with open(filename,'w+') as f:
            f.write(text)
        print(f'File  successfully saved as {filename}')

    return  output



In [ ]:
def pickled_soup(soups, save_location='./', pickle_name='exported_soups.pckl'):
    import pickle
    import sys
    
    filepath = save_location+pickle_name
    
    with open(filepath,'wb') as f:
        pickle.dump(soups, f)
        
    return print(f'Soup successfully pickled. Stored as {filepath}.')

def load_leftovers(filepath):
    import pickle
    
    print(f'Opening leftovers: {filepath}')
    
    with open(filepath, 'rb') as f:
        leftover_soup = pickle.load(f)
        
    return leftover_soup
        

#### Walkthrough - using James' functions

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

from fake_useragent import UserAgent
url = 'https://en.wikipedia.org/wiki/Stock_market'
soup = cook_soup_from_url(url,sleep_time=1)


## Get all links that match are interal wikipedia redirects [yes?]
kwds = {'class':'mw-redirect'}
links = get_all_links(soup)#,kwds)


# preview first 5 links
print(links[:5])

# Turn relative links into absolute links
abs_links = make_absolute_links(url,links)
print(abs_links[:5])

In [ ]:
# Selecting only the first 5 links to test
abs_links_for_soups = abs_links[:5]


# Cooking a batch of soups from those chosen links
batch_of_soups = cook_batch_of_soups(abs_links_for_soups, sleep_time=2)

# batch_of_soups is a list as long as the input link_list
print(f'# of input links: == # of soups in batch:\n{len(abs_links_for_soups)} == {len(batch_of_soups)}\n')

# batch_of_soups is a list of soup-dictionaries
soup_dict = batch_of_soups[0]
print('Each soup_dict has ',soup_dict.keys())

# the page's soup is stored under soup_dict['soup']
soup_from_soup_dict = soup_dict['soup']
type(soup_from_soup_dict)

#### Notes on extracting content.
- Edit the `extract_target_text function` in the James' functions settings or uncomment and use the `extract_target_text_custom function` below

In [ ]:
## ADDING extract_target_text to precisely target text
# def extract_target_text_custom(soup_or_tag,tag_name='p', attrs_dict=None, join_text =True, save_files=False):
#     """User-specified function to add extraction of specific content during 'cook batch of soups'"""
    
#     if attrs_dict==None:
#         found_tags = soup_or_tag.find_all(name=tag_name)
#     else:
#         found_tags = soup_or_tag.find_all(name=tag_name,attrs=attrs_dict)
    
    
#     # if extracting from multiple tags
#     output=[]
#     output = [tag.text for tag in found_tags if tag.text is not None]
    
#     if join_text == True:
#         output = ' '.join(output)

#     ## ADDING SAVING EACH 
#     if save_files==True:
#         text = output #soup.body.string
#         filename =f"drive/My Drive/text_extract_{url_dict_key}.txt"
#         soup_dict['filename'] = filename
#         with open(filename,'w+') as f:
#             f.write(text)
#         print(f'File  successfully saved as {filename}')

#     return  output

# ####################

## RUN A LOOP TO ADD EXTRACTED TEXT TO EACH SOUP IN THE BATCH
for i, soup_dict in enumerate(batch_of_soups):
    
    # Get the soup from the dict
    soup = soup_dict['soup']
    
    # Extract text 
    extracted_text = extract_target_text(soup)
    
    # Add key:value for results of extract
    soup_dict['extracted'] = extracted_text
    
    # Replace the old soup_dict with the new one with 'extracted'
    batch_of_soups[i] = soup_dict
    
example_extracted_text=batch_of_soups[0]['extracted']
print(example_extracted_text[:1000])

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# from fake_useragent import UserAgent
# ua = UserAgent()

# header = {'user-agent':ua.chrome}
# print('Header:\n',header)

# url ='https://en.wikipedia.org/wiki/Stock_market'
# response = requests.get(url, timeout=3, headers=header)

# print('Status code: ',response.status_code)

In [ ]:
### BONUS FUNCTIONS:
- didn't get to in class

def get_all_links(soup):#,attr_kwds=None):
    """Finds all links inside of soup that have the attributes(attr_kwds),which will be used in soup.findAll(attrs=attr_kwds).
    Returns a list of links.
    tag_type = 'a' or 'href'"""
    all_a_tags = soup.findAll('a',attrs=kwds) 
    link_list = []
    for link in all_a_tags:
        test_link = link.get('href')#,attr=kwds)
#         test_link = link.get('href',attrs=kwds)
        link_list.append(test_link)
    return link_list

def make_absolute_links(source_url, rel_link_list):
    """Accepts the source_url for the source page of the rel_link_list and uses urljoin to return a list of valid absolute links."""
    
    from urllib.parse import urlparse, urljoin

    absolute_links=[]

    # Create a for loop to loop through links and make absolute html paths
    for link in rel_link_list:

        # Get base url using a url pasers and the story_url at the beginning of the nb
        abs_link = urljoin(source_url,link)    

        #concatenate and append to a list 
        absolute_links.append(abs_link)
    
    return absolute_links

#### Ex Functions


- `soup = cook_soup_from_url(url)`
    - make a beautiful soup from url
-`soup_links = get_all_links(soup)`
    - get all links from soup and return as a list.
    
-  `absolute_links = make_absolute_links(url, soup_links) `
    - use If `soup_links` are relative links that do not include the website domain and start with '../' instead of 'https://www... ').
    - then can use the `absolute_links` to make new soups to continue searching for your desired content.


def cook_soup_from_url(url, parser='lxml',sleep_time=0):
    """Uses requests to retreive webpage and returns a BeautifulSoup made using lxml parser."""
    import requests
    from time import sleep
    from bs4 import BeautifulSoup
    
    sleep(sleep_time)
    response = requests.get(url)
    
    # check status of request
    if response.status_code != 200:
        raise Exception(f'Error: Status_code !=200.\n status_code={response.status_code}')
                        
    c = response.content
    # feed content into a beautiful soup using lxml
    soup = BeautifulSoup(c,'lxml')
    return soup

def cook_batch_of_soups(link_list, sleep_time=1): #,user_fun = extract_target_text):
    """Accepts a list of links to extract and save in a list of dictionaries of soups
    with their relative url path as their key.
    Set user_fun to None to just extract full soups without user_extract"""
    from time import sleep
    from urllib.parse import urlparse, urljoin

    batch_of_soups = []
    
    for link in link_list:
        soup_dict = {}
        
        
        # turn the url path into the dictionary key/title
        url_dict_key_path = urlparse(link).path
        url_dict_key = url_dict_key_path.split('/')[-1]
        
        soup_dict['_url'] = link
        soup_dict['path'] = url_dict_key

        # make a soup from the current link
        page_soup = cook_soup_from_url(link, sleep_time=sleep_time)
        soup_dict['soup'] = page_soup

        
#         if user_fun!=None:
#             ## ADDING USER-SPECIFIED EXTRACTION FUNCTION       
#             user_output = user_fun(page_soup) #can add inputs to function
#             soup_dict['user_extract'] = user_output
        
        # Add current page's soup to batch_of_soups list
        batch_of_soups.append(soup_dict)
        
    return batch_of_soups


def extract_target_text(soup_or_tag,tag_name='p', attrs_dict=None, join_text =True, save_files=False):
    """User-specified function to add extraction of specific content during 'cook batch of soups'"""
    
    if attrs_dict==None:
        found_tags = soup_or_tag.find_all(name=tag_name)
    else:
        found_tags = soup_or_tag.find_all(name=tag_name,attrs=attrs_dict)
    
    
    # if extracting from multiple tags
    output=[]
    output = [tag.text for tag in found_tags if tag.text is not None]
    
    if join_text == True:
        output = ' '.join(output)

    ## ADDING SAVING EACH 
    if save_files==True:
        text = output #soup.body.string
        filename =f"drive/My Drive/text_extract_{url_dict_key}.txt"
        soup_dict['filename'] = filename
        with open(filename,'w+') as f:
            f.write(text)
        print(f'File  successfully saved as {filename}')

    return  output



def pickled_soup(soups, save_location='./', pickle_name='exported_soups.pckl'):
    import pickle
    import sys
    
    filepath = save_location+pickle_name
    
    with open(filepath,'wb') as f:
        pickle.dump(soups, f)
        
    return print(f'Soup successfully pickled. Stored as {filepath}.')

def load_leftovers(filepath):
    import pickle
    
    print(f'Opening leftovers: {filepath}')
    
    with open(filepath, 'rb') as f:
        leftover_soup = pickle.load(f)
        
    return leftover_soup
        

#### Walkthrough - using James' functions

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

from fake_useragent import UserAgent
url = 'https://en.wikipedia.org/wiki/Stock_market'
soup = cook_soup_from_url(url,sleep_time=1)


## Get all links that match are interal wikipedia redirects [yes?]
kwds = {'class':'mw-redirect'}
links = get_all_links(soup)#,kwds)


# preview first 5 links
print(links[:5])

# Turn relative links into absolute links
abs_links = make_absolute_links(url,links)
print(abs_links[:5])

# Selecting only the first 5 links to test
abs_links_for_soups = abs_links[:5]


# Cooking a batch of soups from those chosen links
batch_of_soups = cook_batch_of_soups(abs_links_for_soups, sleep_time=2)

# batch_of_soups is a list as long as the input link_list
print(f'# of input links: == # of soups in batch:\n{len(abs_links_for_soups)} == {len(batch_of_soups)}\n')

# batch_of_soups is a list of soup-dictionaries
soup_dict = batch_of_soups[0]
print('Each soup_dict has ',soup_dict.keys())

# the page's soup is stored under soup_dict['soup']
soup_from_soup_dict = soup_dict['soup']
type(soup_from_soup_dict)

#### Notes on extracting content.
- Edit the `extract_target_text function` in the James' functions settings or uncomment and use the `extract_target_text_custom function` below

## ADDING extract_target_text to precisely target text
# def extract_target_text_custom(soup_or_tag,tag_name='p', attrs_dict=None, join_text =True, save_files=False):
#     """User-specified function to add extraction of specific content during 'cook batch of soups'"""
    
#     if attrs_dict==None:
#         found_tags = soup_or_tag.find_all(name=tag_name)
#     else:
#         found_tags = soup_or_tag.find_all(name=tag_name,attrs=attrs_dict)
    
    
#     # if extracting from multiple tags
#     output=[]
#     output = [tag.text for tag in found_tags if tag.text is not None]
    
#     if join_text == True:
#         output = ' '.join(output)

#     ## ADDING SAVING EACH 
#     if save_files==True:
#         text = output #soup.body.string
#         filename =f"drive/My Drive/text_extract_{url_dict_key}.txt"
#         soup_dict['filename'] = filename
#         with open(filename,'w+') as f:
#             f.write(text)
#         print(f'File  successfully saved as {filename}')

#     return  output

# ####################

## RUN A LOOP TO ADD EXTRACTED TEXT TO EACH SOUP IN THE BATCH
for i, soup_dict in enumerate(batch_of_soups):
    
    # Get the soup from the dict
    soup = soup_dict['soup']
    
    # Extract text 
    extracted_text = extract_target_text(soup)
    
    # Add key:value for results of extract
    soup_dict['extracted'] = extracted_text
    
    # Replace the old soup_dict with the new one with 'extracted'
    batch_of_soups[i] = soup_dict
    
example_extracted_text=batch_of_soups[0]['extracted']
print(example_extracted_text[:1000])

# import requests
# from bs4 import BeautifulSoup

# from fake_useragent import UserAgent
# ua = UserAgent()

# header = {'user-agent':ua.chrome}
# print('Header:\n',header)

# url ='https://en.wikipedia.org/wiki/Stock_market'
# response = requests.get(url, timeout=3, headers=header)

# print('Status code: ',response.status_code)